<a href="https://colab.research.google.com/github/lkozina1309/tensorflow/blob/main/CNN/Jupyter/transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Script *transfer_learning.py* uses *horses_or_humans* dataset and *MobileNetV2* pretrained model to classify images of horses or humans. 

First, we have to import the libraries.

In [ ]:
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds

Then, we have to download the dataset.


In [ ]:
tfds.disable_progress_bar()

(raw_train, raw_validation, raw_test), metadata = tfds.load(
    'horses_or_humans',
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    with_info=True,
    as_supervised=True,
)

Now, we'll do some preprocessing.

In [ ]:
IMG_SIZE = 160

def format_example(image, label):
  image = tf.cast(image, tf.float32)
  image = (image/127.5) - 1
  image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
  return image, label

In [ ]:
train = raw_train.map(format_example)
validation = raw_validation.map(format_example)
test = raw_test.map(format_example)

In [ ]:
BATCH_SIZE = 32
SHUFFLE_BUFFER_SIZE = 1000

train_batches = train.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
validation_batches = validation.batch(BATCH_SIZE)
test_batches = test.batch(BATCH_SIZE)

First, we'll instantiate a MobileNet V2 model pre-loaded with weights trained on ImageNet

In [ ]:
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)

base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

Set the base model not to be trainable.

In [ ]:
base_model.trainable = False

In [ ]:
model = tf.keras.Sequential([
  base_model,
  tf.keras.layers.GlobalAveragePooling2D(),
  keras.layers.Dense(1)
])

Now, we'll compile our model with low learning rate as we have pretrained weights.

In [ ]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

And now we can train our model.

In [ ]:
history = model.fit(train_batches,
                    epochs=5,
                    validation_data=validation_batches)

We should get almost 100% accuracy on test data.

In [ ]:
loss, accuracy = model.evaluate(test_batches)
print('Test accuracy :', accuracy)